# Faiss

Facebook AI Similarity Search(Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [ ]:
# !pip install faiss-cpu

In [ ]:
# !pip install langchain_community
# # 

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [8]:
loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=200,chunk_overlap=30)
docs=text_splitter.split_documents(documents)

Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200
Created a chunk of size 670, which is longer than the specified 200


Created a chunk of size 984, which is longer than the specified 200
Created a chunk of size 791, which is longer than the specified 200


In [9]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [10]:
print(docs[1])

page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.' metadata={'source': 'speech.txt'}


In [ ]:
embeddings=OllamaEmbeddings(model="gemma2:2b") # if u not mention any model by default llama2 model
db=FAISS.from_documents(docs,embeddings)
db

In [22]:
query="what does the speaker beieve is the main reason the united states should enter the war?"
docs=db.similarity_search(query)
docs  # similarity content will be present.

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [23]:
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'

### AS retriever

We can also convert the vector store into a retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrivers.(retrivers act like as interface)




In [24]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
# later on we work witj diff LLM models that point of time I cannot use vector DB directly but initally i need to convert this 
# db into a reteriver and then finally we will use it.
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'

# Similarity score with score.

There are some FIASS specific methods. One of them is similarity_search_with_score,which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [26]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

# it will calculate the manhattan distance along with sentence

[(Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
  8509.236),
 (Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  9278.529),
 (Document(metadata={'source'

In [27]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.5175815224647522,
 -0.9995439648628235,
 0.2361079752445221,
 -1.3700296878814697,
 -4.315213680267334,
 -0.3562854528427124,
 1.3212285041809082,
 1.00527024269104,
 1.057913899421692,
 -1.1837538480758667,
 -2.0825252532958984,
 0.1377297341823578,
 -2.316490888595581,
 -1.348140001296997,
 -1.0929421186447144,
 -2.2731711864471436,
 0.36353468894958496,
 0.7090752124786377,
 -0.6760249137878418,
 -0.09572767466306686,
 0.39645451307296753,
 -0.0238209031522274,
 -3.704944610595703,
 0.10540825873613358,
 0.13793857395648956,
 -0.9264222979545593,
 0.21075010299682617,
 0.24678558111190796,
 0.5148364901542664,
 -0.011190151795744896,
 -1.863908290863037,
 -0.8779544234275818,
 2.2540531158447266,
 0.9504791498184204,
 0.5161758065223694,
 3.1344947814941406,
 -1.0188250541687012,
 0.07229724526405334,
 2.991046667098999,
 0.3716743588447571,
 -1.9196267127990723,
 0.319240003824234,
 -0.3032168745994568,
 -1.6549763679504395,
 -3.9829671382904053,
 -2.272693634033203,
 1.03406429

In [28]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [29]:
### Saving and loading

db.save_local("faiss_index")

In [30]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [31]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content